# LSTM CAMELS-ES: comparativa
***

***Autor:** Jesús Casado Rodríguez*<br>
***Fecha:** 16-09-2023*<br>

**Introducción:**<br>
En este _notebook_ se comparan los distintos modelos LSTM entrenados sobre los datos de CAMELS-ES.

**Por hacer**:

In [1]:
import numpy as np
import pandas as pd
import os
import glob
from tqdm.notebook import tqdm
from pathlib import Path
from neuralhydrology.utils.config import Config

os.chdir('../')
from model_utils import *
os.chdir('EFAS5/')

In [6]:
runs = [Path(x) for x in glob.glob('runs/EFAS*') if os.path.isdir(x)]
names = [Config(Path(run) / 'config.yml').experiment_name for run in runs]

compare = pd.DataFrame(index=names, columns=['train', 'validation', 'test'])
for run in tqdm(runs):
    cfg = Config(Path(run) / 'config.yml')
    name = cfg.experiment_name
    # extraer el rendimiento de cada época y muestra
    df = pd.DataFrame(columns=range(1, cfg.epochs + 1))
    for epoch in df.columns:
        try:
            _, df[epoch] = get_results(Path(run), 'validation', epoch=epoch)
        except:
            continue
    
    epoch = df.median().idxmax()
    print('{0:<20}\tmejor época: {1}\t\tKGE = {2:.3f}'.format(name, epoch, df.median().max()))
    
    for period in ['train', 'validation', 'test']:
        _, rend = get_results(Path(run), period, epoch=epoch)
        compare.loc[name, period] = rend.median().values[0]

  0%|          | 0/7 [00:00<?, ?it/s]

EFAS_test1          	mejor época: 3		KGE = 0.747
EFAS_test2          	mejor época: 6		KGE = 0.459
EFAS_test3.1        	mejor época: 3		KGE = 0.682
EFAS_test3.2        	mejor época: 1		KGE = 0.615
EFAS_test3.3        	mejor época: 8		KGE = 0.745
EFAS_test3.4        	mejor época: 6		KGE = 0.751
EFAS_test3.5        	mejor época: 5		KGE = 0.664


In [7]:
compare

,train,validation,test
EFAS_test1,0.800841,0.747094,0.712037
EFAS_test2,0.813124,0.458677,0.449706
EFAS_test3.1,0.762351,0.68199,0.697903
EFAS_test3.2,0.803864,0.61529,0.707814
EFAS_test3.3,0.820202,0.744633,0.732228
EFAS_test3.4,0.891496,0.750842,0.79124
EFAS_test3.5,0.849329,0.663674,0.70751


Escojo como mejor modelo el llamado `CAMELS-ES_hidden128`.